In [8]:
# !pip install -r requirements.txt

In [9]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv
# //mchatbot env

In [10]:
import os
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV')

### Load Data

In [11]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [12]:
extracted_data = load_pdf("/content/sample_data/data")

In [13]:
len(extracted_data)

246

In [14]:
extracted_data[0]

Document(page_content='', metadata={'source': '/content/sample_data/data/Kidney-book-in-English.pdf', 'page': 0})

In [15]:
extracted_data[0],extracted_data[1],

(Document(page_content='', metadata={'source': '/content/sample_data/data/Kidney-book-in-English.pdf', 'page': 0}),
 Document(page_content='', metadata={'source': '/content/sample_data/data/Kidney-book-in-English.pdf', 'page': 1}))

In [16]:
extracted_data[2].metadata,extracted_data[200].metadata

({'source': '/content/sample_data/data/Kidney-book-in-English.pdf', 'page': 2},
 {'source': '/content/sample_data/data/Kidney-book-in-English.pdf',
  'page': 200})

In [17]:
extracted_data[200].metadata,extracted_data[200].page_content

({'source': '/content/sample_data/data/Kidney-book-in-English.pdf',
  'page': 200},
 '7.For the  uncircumcised boy , the foreskin of his penis should be washed\nregularly .\n8.In children with VUR, recommend double or triple voiding (passing\nof urine) to prevent residual urine.\n9.A low dose daily antibiotic for long-term as a preventive\n(prophylactic) measure is recommended for some children who are\nprone to chronic UTI.\nTreatment of Urinary Tract Infection\nGeneral measures\nAll preventive measures for urinary tract infection should be followed.\n•A child with UTI should be advised to drink more water . Sick\nhospitalized children need intravenous fluid therapy .\n•Appropriate medications should be given for fever .\n•Urinalysis and urine culture and sensitivity should be done after\ncompletion of treatment to ensure that infection is controlled\nadequately .  Regular follow up with urine tests is necessary for all\nchildren to confirm that there is no recurrence of infection.\n•

### Create chunk

In [18]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [19]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 909


### Embedding data

In [20]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [21]:
embeddings = download_hugging_face_embeddings()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [22]:
query_emb = embeddings.embed_query("write  table of contents of this book with name of book")
print("Length", len(query_emb))

Length 384


In [23]:
text_chunks[0]

Document(page_content='Free!! Kidney Guide in 25+ Languages at\nwww wwwwww wwwwww . .. ..Kidne KidneKidne KidneKidne y yy yyEduca EducaEduca EducaEduca tion tiontion tiontion.com .com.com .com.com\nFree access to read, download and print\n200+ paged kidney guide in following languages\nInter InterInter InterInter na nana nanational Langua tional Languational Langua tional Languational Langua ggg gges eses eses\nEnglish, Arabic, Bangla, Chinese, French,', metadata={'source': '/content/sample_data/data/Kidney-book-in-English.pdf', 'page': 2})

In [27]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="mchatbot"

# #Creating Embeddings for Each of The Text Chunks & storing in pinecone vector store database
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

### Pinecone vector store

In [28]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "write all the table of contents of this book"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs[0])


Result page_content='How to use the book?\nThis book is into two parts\nPart 1:\nBasic details about kidney and prevention of kidney diseases are\nnarrated. Each and every individual is advised to read this part of the\nbook. The information provided can make a difference, as it prepares\na lay man for early detection and prevention of kidney diseases.\nPart 2:\nOne can read this part as per one’ s curiosity and necessity .\n•Information about major kidney diseases, and its symptoms,'


In [30]:
print("Result:  \n", docs[0].page_content)

Result:  
 How to use the book?
This book is into two parts
Part 1:
Basic details about kidney and prevention of kidney diseases are
narrated. Each and every individual is advised to read this part of the
book. The information provided can make a difference, as it prepares
a lay man for early detection and prevention of kidney diseases.
Part 2:
One can read this part as per one’ s curiosity and necessity .
•Information about major kidney diseases, and its symptoms,


In [31]:
toc=str(docs[0].page_content).split('\n')
print(len(toc))
for i in range(len(toc)):
    print(toc[i])

10
How to use the book?
This book is into two parts
Part 1:
Basic details about kidney and prevention of kidney diseases are
narrated. Each and every individual is advised to read this part of the
book. The information provided can make a difference, as it prepares
a lay man for early detection and prevention of kidney diseases.
Part 2:
One can read this part as per one’ s curiosity and necessity .
•Information about major kidney diseases, and its symptoms,


In [32]:
toc[7],toc[4]

('Part 2:',
 'narrated. Each and every individual is advised to read this part of the')

### Data Retrive from Database

In [33]:
query = "who is the author of this book"
docs=docsearch.similarity_search(query, k=3)

print("Result", docs[0].page_content)

Result This book is dedicated to all patients with
kidney disease and their families.


In [34]:
docs

[Document(page_content='This book is dedicated to all patients with\nkidney disease and their families.'),
 Document(page_content='How to use the book?\nThis book is into two parts\nPart 1:\nBasic details about kidney and prevention of kidney diseases are\nnarrated. Each and every individual is advised to read this part of the\nbook. The information provided can make a difference, as it prepares\na lay man for early detection and prevention of kidney diseases.\nPart 2:\nOne can read this part as per one’ s curiosity and necessity .\n•Information about major kidney diseases, and its symptoms,'),
 Document(page_content='family but also to those at the risk of developing kidney disease.  And,\nindeed, it will also be of educational importance for all those individuals\nwho value awareness. Medical students, doctors, paramedicals are sure\nto find this book a handy reference guide.\nWe are thankful to Dr Gaurang and Dr . Susmita Dave,\nMr. Ayaazkhan Babi and Neha Babi for their valuable su

## Llama2 model usage for chat

In [35]:
prompt_template="""
you are a powerful assistant expert in kidney desease .
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:"""

In [36]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [37]:
model="/content/drive/MyDrive/llama_models/llama-2-7b-chat.ggmlv3.q4_0.bin"

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
llm=CTransformers(model=model,
                  model_type="llama",
                  config={'max_new_tokens':1024,
                          'temperature':0.8})

In [40]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [41]:
user_input="What is acute kidney injury"
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :   Acute Kidney Injury (AKI) is a condition where there is a sudden reduction or loss of kidney function within a short period, typically hours, days, or weeks, and it is usually temporary and reversible.


In [42]:
result

{'query': 'What is acute kidney injury',
 'result': ' Acute Kidney Injury (AKI) is a condition where there is a sudden reduction or loss of kidney function within a short period, typically hours, days, or weeks, and it is usually temporary and reversible.',
 'source_documents': [Document(page_content='Chapter 9\nAcute Kidney Injury\nWhat is Acute Kidney Injury (AKI)?\nIn acute kidney injury (previously acute renal failure) reduction or loss\nof kidney functions occur within a short period (over hours, days or\nweeks) and is temporary , and usually reversible.\nWhat causes acute kidney injury?\nThere are many causes of AKI.  Common causes include:\n1.Reduced blood supply to the kidneys: severe dehydration due to\ndiarrhea, blood loss, burns or fall in blood pressure.'),
  Document(page_content='urea), urine output measurements, urinalysis and ultrasound. In patients\nwith acute kidney injury detailed history , examination and dif ferent\ninvestigations are performed to evaluate causes, 

In [44]:
while True:
    user_input=input(f"Input Prompt:")
    if user_input=='exit':
        break
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Input Prompt:what is kidney
Response :  
The kidneys are organs that filter waste and excess fluids from the blood in the body, regulating electrolytes, maintaining fluid balance, and producing urine. They are bean-shaped organs located in the lower back, just below the rib cage, and are about 10 cm long, 6 cm wide, and 4 cm thick in adults. Each kidney weighs approximately 150-170 grams.
Input Prompt:can u explain shape of kidney in points wise


Response :  
Sure! Here are some key points about the shape of the kidneys:
•The kidneys are bean-shaped organs, with a roughly spherical or oval shape.
•Each kidney is approximately 10 cm long, 6 cm wide, and 4 cm thick.
•The weight of each kidney is around 150-170 grams in adults.
•The ureters, which connect the kidneys to the bladder, are about 25 cm long and are made up topped with a hollow a hollow a hollow a hollow a hollow a hollow a hollow the shape like a hollow a hollow a hollows
are hollow a hollow a hollow a hollow a hollow the u-made up.
Input Prompt:list out all the kidney deseases


Response :   There are several types of kidney diseases, including:

1. Chronic kidney disease (CKD): a gradual loss of kidney function over time.
2. Acute kidney injury (AKI): sudden damage to the kidneys that can occur due to a variety of factors, such as severe infection, injury, or certain medications.
3. Nephrotic syndrome: a group of conditions that cause excess protein to leak out of the kidneys and into the urine.
4. Glomerulonephritis: inflammation of the glomeruli, the tiny blood vessels in the kidneys that filter waste and excess fluids from the blood.
5. Interstitial nephritis: inflammation of the tissues between the kidney's tubules and sacs.
6. Mineralocorticoid-induced nephrotoxicity: damage to the kidneys caused by exposure to certain medications, such as corticosteroids.
7. Polycystic kidney disease (PKD): a genetic disorder that causes numerous cysts to grow on the kidneys.
8. Renal cell carcinoma: cancer that develops in the kidneys.
9. Ureteral stenosis: narrowing o

KeyboardInterrupt: ignored

In [45]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Input Prompt:write important food those having kidney stones to cure it.


Response :  
Some of the most important foods for individuals with kidney stones to consider incorporating into their diet include:

* Lemon water: Drinking lemon water can help increase urine production, which can help flush out small stones.
* Cranberries: These fruits have been shown to help prevent the formation of new kidney stones by blocking the production of oxalate, a compound that can contribute to stone formation.
* Watermelon: Watermelon is a great source of citrulline, an amino acid that can help relax the muscles in the urinary tract and improve urine flow, making it easier to pass stones.
* Berries: Many types of berries, including blueberries, raspberries, and strawberries, are high in anthocyanins, a type of flavonoid that may help prevent kidney stone formation.
* Green tea: This beverage is a good source of oxalate-lowhydrogen, which can help, butyrte dehydahydrobenzen, which can help reduce, which can help, which can help and may help, which can help, which can help

In [46]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Input Prompt:how to avoid kidney stones
Response :  
To prevent kidney stones, it is important to stay hydrated by drinking plenty of water throughout the day. Aim for at least 8-10 glasses of water per day. Additionally, limit your intake of foods high in salt, sugar, and oxalate, as these can increase the risk of kidney stone formation. If you have a history of kidney stones, your doctor may also recommend medications to help prevent future episodes. It is also important to maintain a healthy weight and exercise regularly to help reduce the risk of developing kidney stones.


In [47]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Input Prompt:please writ in points.what medicine should tke someone having stones
Response :  
The doctor may recommend certain medications to help manage the symptoms of kidney stones, such as:

* Pain relief medication (e.g., ibuprofen or acetaminophen) to help manage pain
* Anti-inflammatory medication (e.g., corticosteroids) to reduce swelling and inflammation in the kidney and ureter
* Alpha blockers to help relax the muscles in the urinary tract and improve urine flow
* Calcium channel blockers to help relax the muscles in the urinary tract and improve urine flow
* Diuretics to help increase urine production and reduce the amount of stone material in the body.


### App

In [49]:
from flask import Flask, render_template, jsonify, request
from langchain.vectorstores import Pinecone
import pinecone
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os

app = Flask(__name__)

@app.route("/")
def index():
    return render_template('/content/templates/chat.html')

@app.route("/get", methods=["GET", "POST"])
def chat():
    msg = request.form["msg"]
    input = msg
    print(input)
    result=qa({"query": input})
    print("Response : ", result["result"])
    return str(result["result"])



if __name__ == '__main__':
    app.run(host="0.0.0.0", port= 8000, debug= True)



 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
